In [13]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device

In [16]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
# cfg = Config.fromfile('./configs/swin/retinanet_swin-t-p4-w7_fpn_1x_coco.py')


root='../../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash_Ko'

cfg.model.roi_head.bbox_head.num_classes = 10
# cfg.model.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

In [17]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.09s)
creating index...
index created!


In [18]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 4883, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3965  | 1 [Paper]     | 6352  | 2 [Paper pack]  | 897   | 3 [Metal]   | 936   | 4 [Glass]    | 982   |
| 5 [Plastic]       | 2943  | 6 [Styrofoam] | 1263  | 7 [Plastic bag] | 5178  | 8 [Battery] | 159   | 9 [Clothing] | 468   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [19]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2024-01-04 22:41:43,358 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2024-01-04 22:41:43,360 - mmcv - INFO - load model from: torchvision://resnet50
2024-01-04 22:41:43,361 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet50
2024-01-04 22:41:43,578 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2024-01-04 22:41:43,594 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2024-01-04 22:41:43,659 - mmcv - INFO - initialize RetinaHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01, 'override': {'type': 'Normal', 'name': 'retina_cls', 'std': 0.01, 'bias_prob': 0.01}}
2024-01-04 22:41:43,705 - mmcv - INFO - 
backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from torchvision://resnet50 
 
2024-01-04 22:41:43,706 - mmcv - INFO

In [20]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=False)

2024-01-04 22:41:49,171 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2024-01-04 22:41:49,174 - mmdet - INFO - Start running, host: root@instance-5190, work_dir: /data/ephemeral/home/level2-objectdetection-cv-10/develop/Ko/mmdetection/work_dirs/faster_rcnn_r50_fpn_1x_trash_Ko
2024-01-04 22:41:49,175 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_N

2024-01-04 22:41:58,003 - mmdet - INFO - Epoch [1][50/1221]	lr: 1.978e-03, eta: 0:42:56, time: 0.176, data_time: 0.052, memory: 2098, loss_cls: 1.1326, loss_bbox: 0.6754, loss: 1.8080, grad_norm: 0.4894
2024-01-04 22:42:04,605 - mmdet - INFO - Epoch [1][100/1221]	lr: 3.976e-03, eta: 0:37:24, time: 0.132, data_time: 0.007, memory: 2098, loss_cls: 1.1281, loss_bbox: 0.6489, loss: 1.7770, grad_norm: 0.7617
2024-01-04 22:42:11,174 - mmdet - INFO - Epoch [1][150/1221]	lr: 5.974e-03, eta: 0:35:26, time: 0.131, data_time: 0.007, memory: 2098, loss_cls: 1.0514, loss_bbox: 0.6453, loss: 1.6966, grad_norm: 2.1679
2024-01-04 22:42:17,807 - mmdet - INFO - Epoch [1][200/1221]	lr: 7.972e-03, eta: 0:34:28, time: 0.133, data_time: 0.007, memory: 2098, loss_cls: 1.0541, loss_bbox: 0.6344, loss: 1.6885, grad_norm: 2.9003
2024-01-04 22:42:24,440 - mmdet - INFO - Epoch [1][250/1221]	lr: 9.970e-03, eta: 0:33:51, time: 0.133, data_time: 0.007, memory: 2098, loss_cls: 1.0865, loss_bbox: 0.6331, loss: 1.7196,